A critical tool in the synthetic biology design toolbox is strain screening. 
We subject libraries of organisms with designed genetic diversity (a mixed population) to a process relevant to the final goal and select hits based on key metrics.
on the order of thousands and millions, quantity is chosen over quality. The assays are quick and paired with cell sorting tools.
Additional analysis can tie genotypes to the assay results.
For example, a library of 10,000 diversity is sorted with a cytometer using a protein binding dye for fluorescence intensity. Higher fluor intensity correlates with higher protein production

my work has focussed on libary screens on the order of 10s to 100s. Discrete screening, where individual genotypes are screened separately using the system of choice, present the opposite problem.
our measurement is getting closer to the thing we actually care about, product titer, an actual concentration of protein per unit volume or cell count.
Often these efforts lead to a production effort, where a compound of choice actually must be manufactured. Economic analysis must include process yields. 

At this scale, the assays become more expensive and time consuming, and sample replicates are costly.

As we move up in scale in high-throughput workflows, the demands on the screening platform remain (screen many strains), but the cost and time to do so becomes prohibitive.
Experiments regularly lack replicate samples (N=1?) and resources are spent on candidates that are not improvements over the control. 
Several times, projects have ended with the question, did we make it better?

Bayesian inference was introduced as an alternative to the frequentist methods we were using (hypothesis testing with p-values, confidence intervals, multifactor ANOVA). Is there something in baysian methodology that may provide greater insights into the data we have.

Defining the objective of strain screening and problems below...

Objective:
1. Test a population of targets (enzymes, strain genotypes, proteins, small molecules) to find individuals that exhibit optimal performance indicators. 
    Binding affinity, production titer (concentration), economics, growth duration, etc.
2. Performance indicators at all stage of the process should link as close as possible to the final measurement of interest (reactor yield, performance in the clinic, economics)

Questions a screening platform is answering?
1. How do I know if I have a hit?
    what difference am I trying to detect?
    what binding affinity correlates to realistic clinical outcomes? How much material must be made per unit time for the product to be cost effective?
2. What is the minimum number of samples necessary to find a hit?
    Measurements are costly (time and financially)
    multi-armed bandid problem: https://nbviewer.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter6_Priorities/Ch6_Priors_PyMC2.ipynb
3. 

In [1]:
from scipy import stats
import arviz as az
import pymc as pm
import pytensor

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [ ]:
sns.set_style('white')
rng = np.random.default_rng(seed=0)

In [ ]:
screening_data = pd.read_csv('data/screening_sample.csv')
screening_data = screening_data.dropna()
screening_data['Strain ID'] = screening_data['Strain ID'].astype(str)

screening_data = screening_data.drop(screening_data.index[(screening_data['Strain Comment'] == 'Negative Control 1')])

print(screening_data.dtypes)

screening_data = screening_data.drop(screening_data.index[screening_data['Strain Comment'] == 'Negative Control 1'])

data_size = len(screening_data)
print(f'data length: {data_size}')
screening_data.head()

In [ ]:
plot_data = screening_data.sort_values(by=['Strain Comment', 'Strain ID'])

sns.catplot(data=plot_data, x='Strain ID', y='Main %', jitter=False, height=3, aspect=2)
sns.catplot(data=plot_data, x='Strain ID', y='Main %', hue='Strain Comment', dodge=False, kind='box', height=4, aspect=2)

In [ ]:
sns.kdeplot(screening_data, x='Main %', hue='Strain Comment')

In [ ]:
az.plot_kde(screening_data['Main %'], label='Screening Data', rug=False)
screening_data['Main %'].describe()

In [ ]:
#  Setting the model up splitting up the strain IDs
strains = pd.Categorical(screening_data['Strain Comment'])
model_shape = len(strains.categories)

# priors of model parameters mu and sigma
mu_m = screening_data['Main %'].mean()
mu_sd = screening_data['Main %'].std() * 2

sigma_low = 10**-1
sigma_high = 30

draws = 100
chains = 2

coords = {
    'mu_dim_0': strains.categories,
    'sigma_dim_0': strains.categories
    }

with pm.Model(coords=coords) as model:
    mu = pm.Normal('mu', mu=mu_m, sigma=mu_sd, shape=model_shape)
    sigma = pm.Uniform('sigma', lower=sigma_low, upper=sigma_high, shape=model_shape)
    
    main_percent = pm.Normal('Main %', mu=mu[strains.codes], sigma=sigma[strains.codes], observed=screening_data['Main %'])
    
    idata = pm.sample_prior_predictive()
    idata.extend(pm.sample(draws=draws, tune=1000, chains=chains))
    pm.sample_posterior_predictive(idata, extend_inferencedata=True)

In [ ]:
az.style.use('arviz-darkgrid')
az.plot_trace(idata, compact=False, rug=False, divergences=None)

In [ ]:
az.plot_ppc(idata, observed=True, mean=True, group='prior', alpha=0.4)

az.plot_trace(idata, divergences=None)
az.plot_posterior(idata, kind='kde')

az.plot_ppc(idata, kind='kde')

In [ ]:
screening_data['Strain ID'].unique()

In [ ]:
# splitting control data and experimental strains
control_data = screening_data.loc[screening_data['Strain Comment'] == 'Control']
exp_data = screening_data.loc[screening_data['Strain Comment'] == 'DoE']

#  Setting the model up splitting up the strain IDs
exp_strains = pd.Categorical(exp_data['Strain ID'])
exp_model_shape = len(exp_strains.categories)

# priors of model parameters mu and sigma
mu_m = screening_data['Main %'].mean()
mu_sd = screening_data['Main %'].std() * 2

sigma_low = 10**-1
sigma_high = 30

draws = 1000
chains = 4

coords = {
    'mu_dim_0': strains.categories,
    'sigma_dim_0': strains.categories
    }

with pm.Model() as model:
    mu = pm.Normal('mu', mu=mu_m, sigma=mu_sd, shape=model_shape)
    sigma = pm.Uniform('sigma', lower=sigma_low, upper=sigma_high, shape=model_shape)
    
    main_percent = pm.Normal('Main %', mu=mu[strains.codes], sigma=sigma[strains.codes], observed=screening_data['Main %'])

The control data by itself produces a poorly defined distribution. 
For this first analysis we'll look at the entire popultion as as single distribution to sample from.

Next we'll try modeling the control using several distributions and see how that changes the analysis. Basically matching the p-values we are calculating currently